# 调用自定义语音识别API样例

本Notebook提供了两种方式来调用您本地部署的语音识别API：
1.  **HTTP请求 (`curl`)**: 直接在Notebook中执行shell命令。
2.  **Python请求 (`requests`)**: 使用Python的`requests`库来调用API。

---

## 准备工作

在运行下面的代码之前，请确保：
1.  您的语音识别API服务正在 `http://localhost:8000` 上运行。
2.  您已经替换了代码中的 `your-secret-api-key` 为您的真实API密钥。
3.  我们将在本地创建一个临时的WAV文件用于测试。

In [1]:
import os

# 注意：这是一个空的wav文件，仅用于演示API调用。实际使用时，请替换为真实的音频文件。
file_path = 'test_audio.wav'
with open(file_path, 'w') as f:
    f.write('RIFF....WAVEfmt ....') # 一个虚拟的wav文件头

print(f"创建了一个虚拟的测试文件：{os.path.abspath(file_path)}")

创建了一个虚拟的测试文件：D:\PycharmProjects\Dialect-Backend\test_audio.wav


---

## 1. HTTP请求 (`curl`)

(无法在notebook中直接运行，但可以在终端或命令行中执行)

In [2]:
%%bash
# 注意：请将 path/to/your/test_audio.wav 替换为上面创建的文件的绝对路径
# 或者，如果您在与此Notebook相同的目录中运行，可以直接使用 test_audio.wav
FILE_PATH="test_audio.wav"

curl -X POST "http://localhost:8000/v1/audio/transcriptions" \
     -H "Authorization: Bearer your-secret-api-key" \
     -H "Content-Type: multipart/form-data" \
     -F "file=@$FILE_PATH" \
     -F "model=paraformer-zh" \
     -F "response_format=verbose_json" \
     -F "prompt=阿里巴巴,通义千问"

Couldn't find program: 'bash'


---

## 2. Python请求 (`requests`)

这是在Python应用程序中与API交互的推荐方法。我们需要使用`requests`库，它简化了HTTP请求的处理。 [2]

如果您的环境中没有安装`requests`库，请先运行 `pip install requests`。

In [3]:
import requests
import json

# API的URL
url = "http://localhost:8000/v1/audio/transcriptions"

# 替换为您的API密钥
api_key = "your-secret-api-key"

# 设置请求头
headers = {
    "Authorization": f"Bearer {api_key}"
}

# 准备要发送的表单数据
data = {
    'model': 'paraformer-zh',
    'response_format': 'verbose_json',
    'prompt': '阿里巴巴,通义千问'
}

# 准备要上传的文件
# 'rb'表示以二进制只读模式打开文件 [5]
file_path = 'test_audio.wav'
try:
    with open(file_path, 'rb') as f:
        files = {
            'file': (os.path.basename(file_path), f, 'audio/wav')
        }

        # 发送POST请求
        # requests库会自动处理 multipart/form-data 的编码 [3, 5]
        response = requests.post(url, headers=headers, data=data, files=files)

        # 检查响应状态码
        response.raise_for_status()  # 如果状态码不是2xx，将引发HTTPError

        # 以JSON格式打印响应内容
        print(json.dumps(response.json(), indent=2, ensure_ascii=False))

except FileNotFoundError:
    print(f"错误：找不到文件 '{file_path}'")
except requests.exceptions.RequestException as e:
    print(f"请求失败：{e}")
finally:
    # 清理创建的虚拟文件
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"\n已删除测试文件：{file_path}")

请求失败：HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /v1/audio/transcriptions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001CDE891E040>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))

已删除测试文件：test_audio.wav
